# Récupération des données avec l'API Fred

In [1]:
import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import json
#import pour l'API
import requests
from statistics import mean
import time

Création d'un dictionnaire contenant tous les états et des sous-dictionnaires pour chaque comtés avec l'emploi (et pas la louisiane, sera ajoutée après)

76 min d'execution à cause de la limite de requests

In [ ]:
#définition de l'url et des paramètres de la première requête
api_key = "180de2e6a1d1e953d270ebf38341cd44"
param = {"api_key" : api_key, "file_type" : "json", "category_id" : "27281"}
url = "https://api.stlouisfed.org/fred/category/children?"
response = requests.get(url, params=param)

# Check if the request was successful (status code 200)
if response.status_code == 200:
    data = response.json()
    # Process the data as needed
else:
    print(f"Error: {response.status_code}, {response.text}")

# création d'un dictionnaire avec les états pour clés et leur catégorie counties pour valeur
liste = data["categories"] #on rentre dans le premier dictionnaire
dico_id_states = dict()

for dic in liste : #on parcourt la liste en stockant dans un dictionnaire le nom de l'état et le numéro de la catégorie associée
    dico_id_states[dic["name"]] = dic["id"]

state_relous = ["Alaska", "Louisiana","Virgin Islands"] #on retire les états qui n'ont pas de comties
for state in state_relous :
    del dico_id_states[state]

for state in dico_id_states.keys() : #on parcourt les états
    param["category_id"] = dico_id_states[state] #on ajuste les paramètres de la request pour demander la bonne catégorie
    response = requests.get(url, params = param)
    data = response.json()
    dic = data["categories"][0] #on rentre dans le dictionnaire (toujours le premier, c'est celui qui contient les comtés)

    category_id = dic["id"] #on suavegarde la catégorie contenant les comtés de l'état
    dico_id_states[state] = category_id
    time.sleep(0.5)

dic_states_counties = dict() #on initialise un nouveau dictionnaire, qui sera de la forme : {etat1 : {comté1 : {}, comté2 :}, état2 : ....}

for state in dico_id_states.keys() : #on parcourt les états
    param["category_id"] = dico_id_states[state] #on ajuste les paramètres de la request pour demander la bonne catégorie
    response = requests.get(url, params = param)
    data = response.json()
    liste = data["categories"] #on obtient la liste des comtés de l'état

    dic_states_counties[state] = dict() #on prépare le dictionnaire qui contiendra aura les comtés de l'état pour clés
    

    for dic in liste : #on parcourt les dictionnaires (1 par county)
        county_name = dic["name"] #on récupère le nom du county
        dic_states_counties[state][county_name] = {"id" : dic["id"]} #on crée une entrée dans le dic pour le county et on sauvegarde sa catégorie

counties_df = pd.DataFrame() #on  créé le dataframe qui servira à tout sauvegarder et à convertir en csv


for state in dic_states_counties.keys(): #on va parcourir état par état (car c'est les premières clés)
    dic_state = dic_states_counties[state] #on se place dans le dictionnaire de l'état

    for county in dic_states_counties[state].keys(): #on parcourt les counties
        dic_county = dic_state[county] #on se place dans le dictionnaire du county

        url = "https://api.stlouisfed.org/fred/category/series?"
        param = {"api_key" : api_key, "file_type" : "json", "category_id" : dic_county["id"]} #on va accéder à la page contenant les séries du county

        response = requests.get(url, params= param)
        data = response.json()     

        #maintenant on cherche la série dont le titre contient "unemployment rate" et dont la fréquence est annuelle
        liste_series = data["seriess"]
        
        for serie in liste_series: #on parcourt les séries,en cherchant celle qui parle du taux de chomage annuel
            if ("Unemployment Rate" in serie["title"]) and ("Monthly" in serie["frequency"]):
                dic_county["unemployment_rate_id"] = serie["id"]  #on récupère l'id de la série et on le stock
        time.sleep(0.5)

    for county in dic_states_counties[state].keys(): #on parcourt les counties
        dic_county = dic_state[county] #on se place dans le dictionnaire du county

        #récupération de la série
        if "unemployment_rate_id" in dic_county.keys() :

            url = "https://api.stlouisfed.org/fred/series/observations?" #on va chercher les séries correspondant au chomage mensuel
            param = {"api_key" : api_key, "file_type" : "json", "series_id" : dic_county["unemployment_rate_id"], "observation_start" : "2013-01-01", "observation_end" : "2018-03-01"} 
        
            response = requests.get(url, params= param)
            data = response.json()

            liste_observations = [float(obs["value"]) for obs in data["observations"]] #on récupère toutes les observations pour en faire la moyenne sur la période
            
            unemployment_rate_mean = mean(liste_observations)

            dic_county["state"] = state
            dic_county["unemployment_rate"] = unemployment_rate_mean

        time.sleep(0.5)

    df_temp = pd.DataFrame.from_dict(dic_states_counties[state], orient = "index")
    counties_df = pd.concat([counties_df, df_temp])

counties_df.to_csv(path_or_buf="counties_data2.csv")

Avec le code précédent, on obtient un csv contenant les comtés en index, une colonne pour l'id de la catégorie du comté, une colonne pour l'id de la série sur le taux de chômage, une sur le taux moyen de chômage entre janvier 2013 et mars 2018 et enfin une avec l'état dans lequel est situé le comté. Il n'y a pas les parish de Louisiane. Il n'y pas l'Alaska ni les îles vierges.

Ce fichier csv est enregistré et servira de base de travail plus tard, plutôt que rééxécuter le code (qui prend plus d'1h à tourner à cause des limites de requests par minutes).

On va maintenant ajouter les parish de louisiane (on aurait pu les ajouter dès le départ, mais pas grave).

In [ ]:
# on récupère le csv précédent
url="https://drive.google.com/file/d/1zKdZxFTbrolYaEK9EzadOUmT4N9XWXSt/view?usp=drive_link"
url="https://drive.google.com/uc?export=download&confirm=1&id=" + url.split("/")[-2]
county_data_no_louisiana_df = pd.read_csv(url)
county_data_no_louisiana_df.rename(columns={"Unnamed: 0" : "county_name", "id" : "category_id"}, inplace=True)
county_data_no_louisiana_df.set_index("county_name", inplace=True)

county_data_df = county_data_no_louisiana_df

In [ ]:
dic_louisiana = dict() #création du dictionnaire 

# on va se placer dans la catégorie de la louisiane
api_key = "180de2e6a1d1e953d270ebf38341cd44"
url = "https://api.stlouisfed.org/fred/category/children?"
param = {"api_key" : api_key, "file_type" : "json", "category_id" : "28461"} #28461 est la catégorie des parish de la louisiane

response = requests.get(url, params=param)
data = response.json()
liste_parish = data["categories"] #on rentre dans la liste des dicionnaires de la forme {'id': 28462, 'name': 'Acadia Parish, LA', 'parent_id': 28461}, 1 par parish

for parish_dic in liste_parish: #on parcourt les dictionnaires 1 par 1
    dic_louisiana[parish_dic["name"]] = {"category_id" : parish_dic["id"], "state" : "Louisiana"} #on créé dans notre dictionnaire une entrée au nom du parish et avec les données qui nous intéressent

# Récupération de l'id de la série du taux de chomage pour chaque parish
url = "https://api.stlouisfed.org/fred/category/series?"
param = {"api_key" : api_key, "file_type" : "json"}

for parish in dic_louisiana.keys(): #on parcourt les parish
    parish_dic = dic_louisiana[parish] # pour accéder facilement au dictionnaire du parish

    param["category_id"] = parish_dic["category_id"]
    response = requests.get(url, params = param)    
    data = response.json()     

    #maintenant on cherche la série dont le titre contient "unemployment rate" et dont la fréquence est annuelle
    liste_series = data["seriess"]
        
    for serie in liste_series: #on parcourt les séries,en cherchant celle qui parle du taux de chomage annuel
        if ("Unemployment Rate" in serie["title"]) and ("Monthly" in serie["frequency"]):
            parish_dic["unemployment_rate_id"] = serie["id"]  #on récupère l'id de la série et on le stock
    time.sleep(0.5) #pause pour pas dépasser le nombre de requêtes par min



# Récupération du taux de chomage moyen entre janvier 2013 et mars 2018 inclus
url = "https://api.stlouisfed.org/fred/series/observations?"
param = {"api_key" : api_key, "file_type" : "json"}

for parish in dic_louisiana.keys(): #on parcourt les parish
    parish_dic = dic_louisiana[parish] # pour accéder facilement au dictionnaire du parish

    if "unemployment_rate_id" in parish_dic.keys() :

            param = {"api_key" : api_key, "file_type" : "json", "series_id" : parish_dic["unemployment_rate_id"], "observation_start" : "2013-01-01", "observation_end" : "2018-03-01"} 
        
            response = requests.get(url, params= param)
            data = response.json()

            liste_observations = [float(obs["value"]) for obs in data["observations"]] #on récupère toutes les observations pour en faire la moyenne sur la période
            
            unemployment_rate_mean = mean(liste_observations)
            parish_dic["unemployment_rate"] = unemployment_rate_mean

    county_data_df.loc[parish] = parish_dic #on ajoute le parish au dataframe


In [ ]:
county_data_df.to_csv("county_data_complete.csv") #on sauvegarde les donnéees complètes

In [5]:
#On récupère le csv précédemment enregistré (contient unemployement):
url="https://drive.google.com/file/d/1RWRfunmX_Vu_XLoxbEzmCrB27FPc0wbO/view?usp=drive_link"
url="https://drive.google.com/uc?export=download&confirm=1&id=" + url.split("/")[-2]
county_data_df = pd.read_csv(url)
county_data_df.set_index("county_name", inplace=True)

Récupération de la population par county / parish

In [7]:
# Récupération de l'id de la série Resident Population pour chaque county
api_key = "180de2e6a1d1e953d270ebf38341cd44"
url = "https://api.stlouisfed.org/fred/category/series?"
param = {"api_key" : api_key, "file_type" : "json"}

for county, row in county_data_df.iterrows(): #on itère sur chaque county
    param["category_id"] = row["category_id"]
    response = requests.get(url, params=param)
    data = response.json()     

    #maintenant on cherche la série dont le titre contient "unemployment rate" et dont la fréquence est annuelle
    liste_series = data["seriess"]
        
    for serie in liste_series: #on parcourt les séries,en cherchant celle qui parle du taux de chomage annuel
        if ("Resident Population" in serie["title"]) and ("Annual" in serie["frequency"]):
            county_data_df.loc[county, "resident_population_id"] = serie["id"]  #on récupère l'id de la série et on le stock
    time.sleep(0.5) #pause pour pas dépasser le nombre de requêtes par min


#Récupération des données de la série
url = "https://api.stlouisfed.org/fred/series/observations?"
param = {"api_key" : api_key, "file_type" : "json"}

for county, row in county_data_df.iterrows():
    if row["resident_population_id"] != "nan":

        param = {"api_key" : api_key, "file_type" : "json", "series_id" : row["resident_population_id"], "observation_start" : "2013-01-01", "observation_end" : "2017-01-01"}
        response = requests.get(url, params=param)
        data=response.json()

        liste_observations = [float(obs["value"]) for obs in data["observations"]] #on récupère toutes les observations pour en faire la moyenne sur la période
        
        if len(liste_observations) > 0 : #pas présent à l'origine, ce qui a causé un problème
            resident_population_mean = mean(liste_observations)
            county_data_df.loc[county, "resident_population"] = resident_population_mean
        
        time.sleep(0.5)

StatisticsError: mean requires at least one data point

Problème à partir de "Clifton Forge City, VA" car la série est arrêtée, la moyenne ne peut donc pas se faire...

De plus les villes de viriginia n'ont pas de colonne état --> devait être ailleurs, pas grave car leur comté est quand même dedans

On va repartir de là où ca s'est arrêté

In [60]:
url = "https://api.stlouisfed.org/fred/series/observations?"
param = {"api_key" : api_key, "file_type" : "json"}

for county, row in county_data_df.loc["Clifton Forge City, VA":,].iterrows():
    if row["resident_population_id"] != "nan":

        param = {"api_key" : api_key, "file_type" : "json", "series_id" : row["resident_population_id"], "observation_start" : "2013-01-01", "observation_end" : "2017-01-01"}
        response = requests.get(url, params=param)
        data=response.json()

        liste_observations = [float(obs["value"]) for obs in data["observations"]] #on récupère toutes les observations pour en faire la moyenne sur la période
        
        if len(liste_observations) > 0 :
            resident_population_mean = mean(liste_observations)
            county_data_df.loc[county, "resident_population"] = resident_population_mean
        
        time.sleep(0.5)
    

In [63]:
county_data_df.to_csv("resident_population.csv")

In [72]:
county_data_df.notna().all(axis=1).sum()

3114

Récupération du median household income

In [75]:
#On récupère le csv précédemment enregistré (contient unemployement):
url="https://drive.google.com/file/d/1neZeKIIxpm0NJ3UKvH_H-bQBLBKbDv8t/view?usp=drive_link"
url="https://drive.google.com/uc?export=download&confirm=1&id=" + url.split("/")[-2]
county_data_df = pd.read_csv(url)

In [82]:
# Récupération de l'id de la série Resident Population pour chaque county
api_key = "180de2e6a1d1e953d270ebf38341cd44"
url = "https://api.stlouisfed.org/fred/category/series?"
param = {"api_key" : api_key, "file_type" : "json"}

for county, row in county_data_df.iterrows(): #on itère sur chaque county
    param["category_id"] = row["category_id"]
    response = requests.get(url, params=param)
    data = response.json()     

    #maintenant on cherche la série dont le titre contient "unemployment rate" et dont la fréquence est annuelle
    liste_series = data["seriess"]
        
    for serie in liste_series: #on parcourt les séries,en cherchant celle qui parle du taux de chomage annuel
        if ("Estimate of Median Household Income" in serie["title"]) and ("Interval" not in serie["title"]):
            county_data_df.loc[county, "median_household_income_id"] = serie["id"]  #on récupère l'id de la série et on le stock
    time.sleep(0.5) #pause pour pas dépasser le nombre de requêtes par min


#Récupération des données de la série
url = "https://api.stlouisfed.org/fred/series/observations?"
param = {"api_key" : api_key, "file_type" : "json"}

for county, row in county_data_df.iterrows():
    if row["median_household_income_id"] != "nan":

        param = {"api_key" : api_key, "file_type" : "json", "series_id" : row["median_household_income_id"], "observation_start" : "2013-01-01", "observation_end" : "2017-01-01"}
        response = requests.get(url, params=param)
        data=response.json()

        liste_observations = [float(obs["value"]) for obs in data["observations"]] #on récupère toutes les observations pour en faire la moyenne sur la période
        
        if len(liste_observations) > 0 :
            median_household_income_mean = mean(liste_observations)
            county_data_df.loc[county, "median_household_income"] = median_household_income_mean
        
        time.sleep(0.5)

In [ ]:
county_data_df.to_csv("median_household_income.csv")